In [261]:
import pandas as pd
import numpy as np

In [262]:
ch1 = pd.read_csv("CH1.csv")
ch2 = pd.read_csv("CH2.csv")
ch3 = pd.read_csv("CH3.csv")
ch4 = pd.read_csv("CH4.csv")

ch1_X = ch1.select_dtypes(exclude = ["int64", "object"]).copy()
ch2_X = ch2.select_dtypes(exclude = ["int64", "object"]).copy()
ch3_X = ch3.select_dtypes(exclude = ["int64", "object"]).copy()
ch4_X = ch4.select_dtypes(exclude = ["int64", "object"]).copy()

ch1_X.drop(["0"], axis = 1, inplace = True)
ch2_X.drop(["0"], axis = 1, inplace = True)
ch3_X.drop(["0"], axis = 1, inplace = True)
ch4_X.drop(["0"], axis = 1, inplace = True)
'''
ch2_X = ch2_X.iloc[:,:250]
ch3_X = ch3_X.iloc[:,:250]
ch4_X = ch4_X.iloc[:,:250]
'''


'\nch2_X = ch2_X.iloc[:,:250]\nch3_X = ch3_X.iloc[:,:250]\nch4_X = ch4_X.iloc[:,:250]\n'

In [263]:
def getIndicesToBeRemoved(data):
    fft = data.select_dtypes("float64").T

    indices = np.where(np.max(fft) == np.inf)[0]
    indices = np.concatenate((indices, np.where(data.isna().any(axis = 1))[0]))

    return indices

In [264]:
data = pd.concat([ch1_X, ch2_X, ch3_X, ch4_X], axis = 1)
i = getIndicesToBeRemoved(data)

In [265]:
Y = ch1_X.drop(list(set(i))).reset_index(drop=True)

ch2_X = ch2_X.drop(list(set(i))).reset_index(drop=True)
ch3_X = ch3_X.drop(list(set(i))).reset_index(drop=True)
ch4_X = ch4_X.drop(list(set(i))).reset_index(drop=True)

In [266]:
from sklearn.decomposition import PCA

def getSizes(variance_threshold):
    ch2_pca = PCA()
    ch2_pca.fit(ch2_X)
    ch3_pca = PCA()
    ch3_pca.fit(ch3_X)
    ch4_pca = PCA()
    ch4_pca.fit(ch4_X)

    return(len(np.where(np.cumsum(ch2_pca.explained_variance_ratio_) < variance_threshold)[0]), len(np.where(np.cumsum(ch3_pca.explained_variance_ratio_) < variance_threshold)[0]),len(np.where(np.cumsum(ch4_pca.explained_variance_ratio_) < variance_threshold)[0]))



In [267]:
count = getSizes(0.999999)

In [268]:
ch2_pca = PCA(n_components=count[0])
ch3_pca = PCA(n_components=count[1])
ch4_pca = PCA(n_components=count[2])

ch2_X = ch2_pca.fit_transform(ch2_X)
ch3_X = ch2_pca.fit_transform(ch3_X)
ch4_X = ch2_pca.fit_transform(ch4_X)

In [269]:
X = np.concatenate([ch2_X, ch3_X, ch4_X], axis = 1)

In [270]:
X.shape

(2590, 5118)

In [271]:
Y.shape

(2590, 5000)

In [272]:
def normalize(data):
    norm = np.reshape(np.linalg.norm(data, axis = 1), (-1,1))

    return data / norm

In [273]:
X = normalize(X)

In [274]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [275]:
x_train.shape

(2072, 5118)

In [276]:
import tensorflow as tf

x_train = tf.constant(x_train, dtype = tf.float16, shape=x_train.shape)
y_train = tf.constant(y_train, dtype = tf.float16, shape=y_train.shape)

x_test = tf.constant(x_test, dtype = tf.float16, shape=x_test.shape)
y_test = tf.constant(y_test, dtype = tf.float16, shape=y_test.shape)

In [283]:


from keras.layers import Dense, Dropout
from keras import Sequential
from keras.losses import MeanSquaredError

model = Sequential([Dense(2000, activation = "relu"),
                    Dense(5000, activation = "relu")])

model.compile(loss = MeanSquaredError(), optimizer="rmsprop", metrics = ["accuracy"])





In [284]:
model.predict(x_test)
model.summary()

17/17 [==============================] - 0s 8ms/step
Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_50 (Dense)            (None, 2000)              10238000  
                                                                 
 dense_51 (Dense)            (None, 5000)              10005000  
                                                                 
Total params: 20,243,000
Trainable params: 20,243,000
Non-trainable params: 0
_________________________________________________________________


In [287]:
model.fit(x_train, y_train, epochs = 10, validation_data = [x_test, y_test])

Epoch 1/10
65/65 [==============================] - 1s 11ms/step - loss: 6.8465e-07 - accuracy: 0.7394 - val_loss: 7.6236e-07 - val_accuracy: 0.7664
Epoch 2/10
65/65 [==============================] - 1s 10ms/step - loss: 6.8421e-07 - accuracy: 0.7428 - val_loss: 7.6513e-07 - val_accuracy: 0.7548
Epoch 3/10
65/65 [==============================] - 1s 10ms/step - loss: 6.8557e-07 - accuracy: 0.7365 - val_loss: 7.6012e-07 - val_accuracy: 0.7683
Epoch 4/10
65/65 [==============================] - 1s 10ms/step - loss: 6.8373e-07 - accuracy: 0.7423 - val_loss: 7.6926e-07 - val_accuracy: 0.7568
Epoch 5/10
65/65 [==============================] - 1s 10ms/step - loss: 6.8485e-07 - accuracy: 0.7346 - val_loss: 7.6311e-07 - val_accuracy: 0.7587
Epoch 6/10
65/65 [==============================] - 1s 10ms/step - loss: 6.8307e-07 - accuracy: 0.7403 - val_loss: 7.7127e-07 - val_accuracy: 0.7587
Epoch 7/10
65/65 [==============================] - 1s 10ms/step - loss: 6.8471e-07 - accuracy: 0.7423 - v